In [1]:
from geopy.geocoders import Nominatim
import pandas as pd

# Paradas

In [42]:
df_paradas = pd.read_csv('../../../data/raw/gps/stops.txt')
df_paradas.drop_duplicates(subset=['stop_id'], inplace=True)
df_paradas.head()

,Unnamed: 0,stop_id,stop_name,stop_lat,stop_lon,adress,district
0,0,10,"AVENIDA CEL CARVALHO, 1180",-3.702286,-38.590602,"Avenida Radialista José Lima Verde, Barra do C...",Barra do Ceará
1,1,1000,"RUA DOR JOAO MOREIRA, 489",-3.722157,-38.528991,"Rua Doutor João Moreira, Centro, Fortaleza, Re...",Centro
2,2,1001,"RUA DOR JOAO MOREIRA, 163",-3.723169,-38.526007,"Museu da Indústria, 143, Rua Doutor João Morei...",Centro
3,3,1010,"AVENIDA IMPERADOR, 550",-3.726137,-38.534206,"Hospital Geral Doutor César Cals, 545, Avenida...",Centro
4,4,1013,"AVENIDA IMPERADOR, 262",-3.723690,-38.533477,"Pacoti Couros, 230, Avenida Imperador, Centro,...",Centro


In [10]:
df_paradas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4796 entries, 0 to 4795
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   stop_id              4796 non-null   int64  
 1   stop_code            0 non-null      float64
 2   stop_name            4796 non-null   object 
 3   stop_desc            0 non-null      float64
 4   stop_lat             4796 non-null   float64
 5   stop_lon             4796 non-null   float64
 6   zone_id              0 non-null      float64
 7   stop_url             0 non-null      float64
 8   location_type        4796 non-null   int64  
 9   parent_station       0 non-null      float64
 10  stop_timezone        0 non-null      float64
 11  wheelchair_boarding  0 non-null      float64
dtypes: float64(9), int64(2), object(1)
memory usage: 487.1+ KB


In [11]:
df_paradas.drop(columns=['stop_code', 'stop_desc', 'zone_id', 'stop_url', 'location_type', 'parent_station', 'stop_timezone', 'wheelchair_boarding'], inplace=True)
df_paradas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4796 entries, 0 to 4795
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   stop_id    4796 non-null   int64  
 1   stop_name  4796 non-null   object 
 2   stop_lat   4796 non-null   float64
 3   stop_lon   4796 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 187.3+ KB


In [14]:
locator = Nominatim(user_agent="myGeocoder")

enderco = []

for index, row in df_paradas.iterrows():
    location = locator.reverse(str(row['stop_lat']) + ',' + str(row['stop_lon']))
    enderco.append(location.address)

df_paradas['adress'] = enderco

In [44]:
a = df_paradas.adress.to_list()[1].address
a = a[0:a.find(', Fortaleza,')]
a = a[a.rfind(', ')+2:len(a)]
a

'Centro'

In [54]:
bairros = []

for index, row in df_paradas.iterrows():
    bairro = row['adress']
    bairro = bairro[0:bairro.find(', Fortaleza,')]
    bairro = bairro[bairro.rfind(', ')+1:len(bairro)]
    bairro = bairro.strip()
    bairros.append(bairro)

df_paradas['district'] = bairros

In [95]:
df_paradas['city'] = "Fortaleza"
df_paradas.head()

,Unnamed: 0,stop_id,stop_name,stop_lat,stop_lon,adress,district,city
0,0,10,"AVENIDA CEL CARVALHO, 1180",-3.702286,-38.590602,"Avenida Radialista José Lima Verde, Barra do C...",Barra do Ceará,Fortaleza
1,1,1000,"RUA DOR JOAO MOREIRA, 489",-3.722157,-38.528991,"Rua Doutor João Moreira, Centro, Fortaleza, Re...",Centro,Fortaleza
2,2,1001,"RUA DOR JOAO MOREIRA, 163",-3.723169,-38.526007,"Museu da Indústria, 143, Rua Doutor João Morei...",Centro,Fortaleza
3,3,1010,"AVENIDA IMPERADOR, 550",-3.726137,-38.534206,"Hospital Geral Doutor César Cals, 545, Avenida...",Centro,Fortaleza
4,4,1013,"AVENIDA IMPERADOR, 262",-3.723690,-38.533477,"Pacoti Couros, 230, Avenida Imperador, Centro,...",Centro,Fortaleza


In [119]:
paradas_fora_fortal = df_paradas[df_paradas.district == 'Brasi'].adress.value_counts().index.to_list()

for i in paradas_fora_fortal:
    cidade = i[:i.find(', Região Geográfica Imediata de Fortaleza,')]
    cidade = cidade[cidade.rfind(',')+1:].strip()
    df_paradas.loc[df_paradas['adress']==i,'city'] = cidade

    bairro = i[:i.find(', ' + cidade + ',')]
    bairro = bairro[bairro.rfind(',')+1:].strip()
    df_paradas.loc[df_paradas['adress']==i,'district'] = bairro
;

''

In [120]:
bairros = df_paradas.district.value_counts().index.tolist()

print(bairros)

['Passaré', 'Centro', 'Prefeito José Walter', 'Mondubim', 'Messejana', 'Jangurussu', 'Parque Dois Irmăos', 'Bom Jardim', 'Parangaba', 'Fátima', 'Vila Velha', 'Barra do Ceará', 'Granja Lisboa', 'Planalto Ayrton Senna', 'Antônio Bezerra', 'Serrinha', 'Edson Queiroz', 'Aldeota', 'Barroso', 'Siqueira', 'Vicente Pinzón', 'Parque Santa Maria', 'Paupina', 'Lagoa Redonda', 'Bonsucesso', 'Engenheiro Luciano Cavalcante', 'Sapiranga / Coité', 'Conjunto Ceará II', 'Vila Uniăo', 'Granja Portugal', 'Jóquei Clube', 'Meireles', 'Jardim das Oliveiras', 'Pedras', 'Benfica', 'Cais do Porto', 'Conjunto Ceará I', 'Cajazeiras', 'Quintino Cunha', 'Cidade dos Funcionários', 'Pici', 'Novo Mondubim', 'Dionísio Torres', 'Henrique Jorge', 'Itaperi', 'Padre Andrade', 'Canindezinho', 'Boa Vista/Castelão', 'São João do Tauape', 'Cambeba', 'Bela Vista', 'Guajeru', 'Jacarecanga', 'José de Alencar', 'Montese', 'Maraponga', 'Dom Lustosa', 'Parque Săo José', 'Papicu', 'João XXIII', 'Cocó', 'Rodolfo Teófilo', 'Joaquim Táv

In [121]:
len(bairros)

130

In [67]:
df_paradas.to_csv('../../../data/processed/paradas.csv', encoding='utf-8')

# Rotas

In [7]:
df_rotas = pd.read_csv('../../../data/raw/gps/routes.txt')
df_rotas.drop_duplicates(subset=['route_id'], inplace=True)
df_rotas.head()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,4,1,4-Messejana/Papicu/Cambeba/TJ,Messejana/Papicu/Cambeba/TJ,NaN,3,NaN,NaN,NaN
1,11,1,11-Circular I,Circular I,NaN,3,NaN,NaN,NaN
2,13,1,13-Aguanambi/Rodoviária I,Aguanambi/Rodoviária I,NaN,3,NaN,NaN,NaN
3,14,1,14-Aguanambi/Rodoviária II,Aguanambi/Rodoviária II,NaN,3,NaN,NaN,NaN
4,15,1,15-Cj Ceará/Antônio Bezerra I,Cj Ceará/Antônio Bezerra I,NaN,3,NaN,NaN,NaN


In [9]:
df_rotas.drop(columns=['agency_id', 'route_desc', 'route_type', 'route_url', 'route_color', 'route_text_color'], inplace=True)
df_rotas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 308 entries, 0 to 307
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   route_id          308 non-null    int64 
 1   route_short_name  308 non-null    object
 2   route_long_name   308 non-null    object
dtypes: int64(1), object(2)
memory usage: 9.6+ KB


In [10]:
df_rotas.to_csv('../../../data/processed/rotas.csv', encoding='utf-8')

# Paradas por rota

In [26]:
df_viagens = pd.read_csv('../../../data/raw/gps/trips.txt')
df_viagens.drop_duplicates(subset=['route_id'], inplace=True)
df_viagens.head()

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible
0,11,D,D011-T01V01B01-I,NaN,NaN,NaN,NaN,shape011-I,2
17,14,D,D014-T01V01B01-V,NaN,NaN,NaN,NaN,shape014-V,2
36,15,D,D015-T01V01B01-I,NaN,NaN,NaN,NaN,shape015-I,1
138,21,D,D021-T01V01B01-I,NaN,NaN,NaN,NaN,shape021-I,2
170,22,D,D022-T01V01B01-I,NaN,NaN,NaN,NaN,shape022-I,2


In [27]:
df_viagens.drop(columns=['service_id', 'trip_headsign', 'trip_short_name', 'direction_id', 'block_id', 'shape_id', 'wheelchair_accessible'], inplace=True)
df_viagens.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269 entries, 0 to 57594
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   route_id  269 non-null    int64 
 1   trip_id   269 non-null    object
dtypes: int64(1), object(1)
memory usage: 6.3+ KB


In [16]:
df_horario_parada = pd.read_csv('../../../data/raw/gps/stop_times.txt')
df_horario_parada.drop_duplicates(inplace=True)
df_horario_parada.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,D011-T01V01B01-I,05:13:00,05:13:00,7038,1,NaN,NaN,NaN,NaN
1,D011-T01V01B01-I,NaN,NaN,1013,2,NaN,NaN,NaN,NaN
2,D011-T01V01B01-I,05:14:00,05:14:00,1015,3,NaN,NaN,NaN,NaN
3,D011-T01V01B01-I,05:15:00,05:15:00,4251,4,NaN,NaN,NaN,NaN
4,D011-T01V01B01-I,05:16:00,05:16:00,990,5,NaN,NaN,NaN,NaN


In [17]:
df_horario_parada.drop(columns=['arrival_time', 'departure_time', 'stop_headsign', 'pickup_type', 'drop_off_type', 'shape_dist_traveled'], inplace=True)
df_horario_parada.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1886957 entries, 0 to 1886956
Data columns (total 3 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   trip_id        object
 1   stop_id        int64 
 2   stop_sequence  int64 
dtypes: int64(2), object(1)
memory usage: 57.6+ MB


In [40]:
df_rota_paradas = df_viagens.merge(df_horario_parada, left_on='trip_id', right_on='trip_id')
df_rota_paradas.drop(columns=['trip_id'], inplace=True)
df_rota_paradas.drop_duplicates(inplace=True)
df_rota_paradas

,route_id,stop_id,stop_sequence
0,11,7038,1
1,11,1013,2
2,11,1015,3
3,11,4251,4
4,11,990,5
...,...,...,...
8940,860,2205,22
8941,860,2206,23
8942,860,5512,24
8943,860,2185,25


In [41]:
df_rota_paradas.to_csv('../../../data/processed/rota_paradas.csv', encoding='utf-8')